# Tensorflow Estimator

In [1]:
# Basic Imports

import tensorflow as tf

import pandas as pd

import numpy as np 

import os

from sklearn.model_selection import train_test_split

In [2]:

# Load the dataset


wines_df = pd.read_csv("../data/winequality.csv")

###########################################

# tf doesn't like spaces in column names so I replace them with _ 

new_col_list = []
for col_name in wines_df.columns:
    new_col_names = col_name.replace(" ", "_")
    new_col_list.append(new_col_names)
wines_df.columns = new_col_list



###########################################

# Change the type of the index values from [0....1598] to [wine_1....wine_1599]

wines_df.index += 1 # add 1 to index values to start wine specification from wine_1 rather than wine_0

index_as_string = wines_df.index.astype('str')

wines_df.index = 'wine_' + index_as_string

##########################################

## Splitting the dataset to training, validation and test sets

In [3]:
# shuffle the data of the wines_df

wines_df = wines_df.sample(frac=1) 

# almost 70% training , 15% validation, 15% test set

intermediate_set, valid_set = train_test_split(wines_df, test_size=0.15) 
train_set, test_set = train_test_split(intermediate_set, test_size=0.15)

In [4]:
train_y = train_set.pop('quality')

In [5]:
valid_y = valid_set.pop('quality')

In [6]:
test_y = test_set.pop('quality')

In [12]:
def json_creator_v2(df=test_set,samples=1):
    '''
    A function that creates a json file containing the test samples in appropriate format for tf serving
    '''
    import json
    
    samples = df.head(samples)
    samples_dict = samples.to_dict(orient='records')
    for elements in samples_dict:
        for key,values in elements.items():
            elements[key] = [values]

        
    json_dict = {}
    json_dict["instances"] = samples_dict
    with open('results.json','w') as f:
        json.dump(json_dict,f, indent=4)

In [7]:
json_creator_v2(samples=4)

In [22]:
train_batch_size = 33
valid_batch_size = 20
test_batch_size = 17

In [7]:
params = {'learning_rate': 0.001,
         'feature_names':['fixed_acidity','volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide','density','pH','sulphates','alcohol'],
         'train_batch_size' : 33,
         'valid_batch_size' : 20,
         'test_batch_size' : 17,
         'number_of_hidden_units': [200,180,10], 
         'num_epochs' : 5,
         }
            


In [18]:
params

{'learning_rate': 0.001,
 'feature_names': ['fixed_acidity',
  'volatile_acidity',
  'citric_acid',
  'residual_sugar',
  'chlorides',
  'free_sulfur_dioxide',
  'total_sulfur_dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol'],
 'train_batch_size': 33,
 'valid_batch_size': 20,
 'test_batch_size': 17,
 'number_of_hidden_units': [200, 180, 10],
 'num_epochs': 5,
 'fixed_acidity': [15.9, 4.6],
 'volatile_acidity': [1.58, 0.12],
 'citric_acid': [1.0, 0.0],
 'residual_sugar': [15.4, 0.9],
 'chlorides': [0.61, 0.012],
 'free_sulfur_dioxide': [72.0, 1.0],
 'total_sulfur_dioxide': [289.0, 6.0],
 'density': [1.00369, 0.99007],
 'pH': [4.01, 2.74],
 'sulphates': [2.0, 0.33],
 'alcohol': [14.9, 8.4]}

## Input functions

In [ ]:
# When to use df.copy


# This will change the original df because of df[0:1]
df = pd.DataFrame({'x': [1,2,3]})
df_sub = df[0:2]
display(df, df_sub)
df_sub.x = -10
print("The original df changed as well")
display(df)
print("As we changed dfsub")
display(df_sub)


# Instead If I create a copy I wont have this problem

print("We create the dataframes again ")

df = pd.DataFrame({'x': [1,2,3]})
df.sub = df[0:2].copy()
display(df, df_sub)
df_sub.x = -10
print("Will the original df change now?")
display(df)
print("Comparing it to dfsub hoping they are different")
display(df_sub)

In [9]:
# for serving input function 

def max_min_finder(train_set, params):
    my_dict = {}
    for columns in params['feature_names']:
        max_value = train_set[columns].max()
        min_value = train_set[columns].min()
        my_dict[columns] = [max_value,min_value]
    return my_dict



In [16]:
params.update(max_min_finder(train_set,params))

In [17]:
print(params)

{'learning_rate': 0.001, 'feature_names': ['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates', 'alcohol'], 'train_batch_size': 33, 'valid_batch_size': 20, 'test_batch_size': 17, 'number_of_hidden_units': [200, 180, 10], 'num_epochs': 5, 'fixed_acidity': [15.9, 4.6], 'volatile_acidity': [1.58, 0.12], 'citric_acid': [1.0, 0.0], 'residual_sugar': [15.4, 0.9], 'chlorides': [0.61, 0.012], 'free_sulfur_dioxide': [72.0, 1.0], 'total_sulfur_dioxide': [289.0, 6.0], 'density': [1.00369, 0.99007], 'pH': [4.01, 2.74], 'sulphates': [2.0, 0.33], 'alcohol': [14.9, 8.4]}


In [15]:
params

{'learning_rate': 0.001,
 'feature_names': ['fixed_acidity',
  'volatile_acidity',
  'citric_acid',
  'residual_sugar',
  'chlorides',
  'free_sulfur_dioxide',
  'total_sulfur_dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol'],
 'train_batch_size': 33,
 'valid_batch_size': 20,
 'test_batch_size': 17,
 'number_of_hidden_units': [200, 180, 10],
 'num_epochs': 5}

In [17]:
df = wines_df.copy()
min_values = df.min(axis=0)
max_values = df.max(axis=0)
max_minus_min = max_values - min_values
df = df - min_values
df = df.div(max_minus_min)

In [34]:
print((df.any() >=0) & (df.any()<=1))

fixed_acidity           True
volatile_acidity        True
citric_acid             True
residual_sugar          True
chlorides               True
free_sulfur_dioxide     True
total_sulfur_dioxide    True
density                 True
pH                      True
sulphates               True
alcohol                 True
quality                 True
dtype: bool


In [23]:
# creating the input function for training, validation, prediction
def my_input_fn(dataframe, labels, batch_size, train):
    
    '''
    dataframe = train_set or valid_ set or test_set
    
    labels = train_y or valid_set or test_set
    
    batch_size = 33 for training or 20 for validation or 17 for test_set   
    '''
    
    # normalization
    df = dataframe.copy()
    min_values = df.min(axis=0)
    max_values = df.max(axis=0)
    max_minus_min = max_values - min_values
    df = df - min_values
    df = df.div(max_minus_min) 
    dataset = tf.data.Dataset.from_tensor_slices((df.values, labels.values))
    if train:
        dataset = dataset.shuffle(batch_size)
    dataset = dataset.batch(batch_size)
    return dataset



In [24]:
data = my_input_fn(dataframe=train_set, labels=train_y, batch_size=train_batch_size, train=True)

In [ ]:
tf.data.Dataset.shuffle

In [ ]:
# Doulepse
logits = []
probabilities = []
predicts = []
lbls = []


for row in data.take(3):

    feat , labels = row
    
    # build model graph 
    inputs = tf.keras.Input(shape=(11,))
    dense_1 = tf.keras.layers.Dense(units=20, input_shape=(33,11), activation='relu', dtype='float32')(inputs)
    dense_2 = tf.keras.layers.Dense(units=10, input_shape=(33,20), activation="softmax", dtype='float32')(dense_1)
    model = tf.keras.Model(inputs=inputs, outputs=dense_2)
    
    

    lbls.append(labels)
    
    # model's output is the logits for every batch
    
    
    
    logit = model(feat)
    
    
    print(logit)
    aha = []
    for j in logit:
        aha.append(sum(j))
    
    print(aha)
    print(len(aha))
    
    break
    logits.append(logit)
    probs = tf.keras.activations.softmax(logit)
    probabilities.append(probs)
    preds = tf.argmax(probs, axis=1, output_type=tf.int64)
    predicts.append(preds)

    
    
    #print(class_preds_per_point)

    # check the first data point
#     print(f"The logits of each class in the first data point are \n {logit[0]}")   
#     print("\n")
#     print(f"The probabilities of each class in the first data point are \n {probs[0,:]}")
#     print("\n")
#     print(f"The index of the most probable class for the first data point is \n {preds[0]}")
#     print("\n")




    loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(labels, tf.cast(probs, tf.float32)))
    # var_list = [dense_1.trainable_weights, dense_2.trainable_weights]
    
    
#     print("The loss is {}".format(loss))
#     print(probs)
#     print("\n")
#     print(preds)
#     print("\n")
#     print(labels)

## Estimator Call

In [ ]:
# model_fn without feature_columns

def model_fn(features, labels, mode, params=params):
    
    
    
    # building the model graph
    inputs = tf.keras.Input(shape=(11,))
    dense_1 = tf.keras.layers.Dense(20, activation='relu', name='dense_1', dtype=tf.float32)(inputs)
    dense_2 = tf.keras.layers.Dense(15, activation='relu', name='dense_2', dtype=tf.float32)(dense_1)
    dense_3 = tf.keras.layers.Dense(10,  activation='softmax', name='dense_3', dtype=tf.float32)(dense_2)
    model = tf.keras.Model(inputs=inputs, outputs=dense_3)
    
    
    
    def loss():
        scce = tf.keras.losses.SparseCategoricalCrossentropy()
        return tf.reduce_mean(scce(labels,probs))
    
    
    def my_acc(labels,probs):    
        '''
        SparseCategoricalAccuracy

        Calculates how often predictions match integer labels

        Example

        label = [3,4]

        probs = [[0.3, 0.1, 0.2, 0.4, 0] , [0.4, 0.2, 0.1, 0, 0.3] match_indx_0] 

        In the first array of probs we have a match between the index with the highest prob (3,0.4)
        and the label 3.

        In the second array of probs we dont have a much since the index with the highest prob is 0

        Therefore sca = # matches / # total labels = 1/2 = 0.50

        '''

        acc_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')
        acc_metric.update_state(y_true=labels, y_pred=probs)
        return {'SparseCategoricalAcc': acc_metric}
    
    
    with tf.GradientTape() as tape:
        tape.watch(model.trainable_variables)
        probs = model(features)


    




        if mode == tf.estimator.ModeKeys.TRAIN:
            optimizer = tf.keras.optimizers.Adam()
            optimizer.iterations = tf.compat.v1.train.get_global_step()
            gradients = tape.gradient(loss(), model.trainable_variables)
            train_op = optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            return tf.estimator.EstimatorSpec(mode, loss=loss(), train_op = train_op)


    
    
    if mode == tf.estimator.ModeKeys.EVAL:
        metrics = my_acc(labels,probs)
        return tf.estimator.EstimatorSpec(mode,loss=loss(), eval_metric_ops=metrics)



    elif mode == tf.estimator.ModeKeys.PREDICT:
        index_preds = tf.argmax(probs,axis=1,output_type=tf.int64)
        predictions = {
            'class_index_predictions' : index_preds,
            'probs_of_classes' : probs
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)




In [ ]:
configurations = tf.estimator.RunConfig(save_summary_steps=35, save_checkpoints_steps=35, keep_checkpoint_max=2)
my_estimator = tf.estimator.Estimator(model_fn = model_fn, model_dir='../model/checkpoints', params=params, config=configurations)

for i in range(params['num_epochs']):
    my_estimator.train(input_fn=lambda:my_input_fn(train_set, train_y, params['train_batch_size'], train=True), steps=35)

In [ ]:
my_estimator.evaluate(input_fn=lambda:input_fn(valid_set, valid_y, params['valid_batch_size'], train=False), steps=12, checkpoint_path='../model/checkpoints/model.ckpt-525')

In [ ]:
preds = my_estimator.predict(input_fn=lambda:input_fn(test_set, test_y, params['test_batch_size'], train=False), yield_single_examples=False)

### train_and_evaluate

In order to use the 

**tf.estimator.train_and_evaluate(
    estimator, train_spec, eval_spec
)**

method we need to supply the specifications for training *train_spec* and evaluation *eval_spec* accordingly.


* tf.estimator.TrainSpec(
    input_fn, max_steps=None, hooks=None)
    
* tf.estimator.EvalSpec(
    input_fn, steps=100, name=None, hooks=None, exporters=None,
    start_delay_secs=120, throttle_secs=600
)



In [ ]:
train_spec = tf.estimator.TrainSpec(input_fn=lambda:my_input_fn(train_set, train_y, train_batch_size, train=True), max_steps=35*NUM_EPOCHS)

eval_spec = tf.estimator.EvalSpec(input_fn=lambda:my_input_fn(valid_set, valid_y, valid_batch_size, train=False), steps=12, start_delay_secs=60, throttle_secs=120 )

for i in range(params['num_epochs']):
    tf.estimator.train_and_evaluate(my_estimator, train_spec, eval_spec)

## Examples of function minimization

In [ ]:
# Example of minimizing a function f(x) = x^2-10x+25 with .minimize


x = tf.Variable(0,dtype=tf.float32, trainable=True)


@tf.function
def f_x():
    return x**2 -10*x + 25



for _ in range(200):
    print([x.numpy(), f_x().numpy()])
    opt = tf.keras.optimizers.SGD(0.01).minimize(f_x,[x])


    
    
    




In [ ]:
# Example of minimizing a function f(x) = x^2-10x+25 with GradientTape


x = tf.Variable([0], dtype=tf.float32, trainable=True)


@tf.function
def f_x():
    return x**2 -10*x +25

opt = tf.keras.optimizers.SGD(0.01)
for _ in range(200):
    print([x[0].numpy(), f_x().numpy()])
    with tf.GradientTape() as tape:
        df_dx = tape.gradient(f_x(),x)
        opt.apply_gradients(zip([df_dx],[x]))
    



In [ ]:
y_true = tf.constant([0,1,1])

In [ ]:
y_pred = tf.constant([[4, 2, 0],[1,5,3],[3,1,1]]) 

In [ ]:
tf.keras.metrics.sparse_categorical_accuracy(y_true,y_pred).numpy()

## Feature Columns

In [ ]:
a = tf.Variable([[1,2,3],
                 [4,5,6]])


In [ ]:
for row in data:
    
    feat, label = row
    
    
    feature_names = ['fixed_acidity','volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide','density','pH','sulphates','alcohol']   
    #feats = tf.split(feat,11,axis=1)
    

    
    
    INPUT_FEATURE = 'x'
    feature_columns =[tf.feature_column.numeric_column(INPUT_FEATURE, shape=(11,))]
    
    feature_layer_inputs = {}  # dictionary that maps feature names to tf.keras.Input layers
    #for names in feature_names:
        #feature_columns.append(tf.feature_column.numeric_column(names))
    feature_layer_inputs[INPUT_FEATURE] = tf.keras.Input(shape=(11,), dtype=tf.float32)
        
    
    
    feature_layer = tf.keras.layers.DenseFeatures(feature_columns,dtype=tf.float32)(feature_layer_inputs)
    dense_1 = tf.keras.layers.Dense(20, activation='relu',name='dense_1',dtype=tf.float32)(feature_layer)
    dense_2 = tf.keras.layers.Dense(10, activation='relu', name='dense_2', dtype=tf.float32)(dense_1)
    model = tf.keras.Model(inputs=feature_layer_inputs, outputs=dense_2)
    
    
    
    logits = model(feat)
    probs = tf.keras.activations.softmax(logits)
    class_index = tf.argmax(probs, axis=1 , output_type=tf.int64)
    
    loss = tf.reduce_sum(tf.keras.losses.sparse_categorical_crossentropy(label,probs))
    #print(loss)
    break

In [25]:
# model_fn with feature_columns 

def my_model_fn(features, labels, mode, params):
    

    # extract those from params dict
#     feature_names = ['fixed_acidity','volatile_acidity', 'citric_acid', 'residual_sugar', 'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide','density','pH','sulphates','alcohol']   
    
    
#     feats = tf.split(features,11,axis=1)
    
#     feature_columns =[]
#     feature_layer_inputs = {}  # dictionary that maps feature names to tf.keras.Input layers
#     for names in feature_names:
#         feature_columns.append(tf.feature_column.numeric_column(names))
#         feature_layer_inputs[names] = tf.keras.Input(dtype=tf.dtypes.float64, shape=(1,), name=names)
    
    
    input_feature = 'x'
    feature_columns =[tf.feature_column.numeric_column(input_feature, shape=(11,))]    
    feature_layer_inputs = { input_feature: tf.keras.Input(shape=(11,), dtype=tf.float32)} 
    
    
    # model's architecture
    
    feature_layer = tf.keras.layers.DenseFeatures(feature_columns,dtype=tf.float64)
    feature_layer_outputs = feature_layer(feature_layer_inputs)
    dense_1 = tf.keras.layers.Dense(220, activation='relu',name='dense_1',dtype=tf.float64)(feature_layer_outputs)
    dense_2 = tf.keras.layers.Dense(350, activation='relu', name='dense_2', dtype=tf.float64)(dense_1)
    dense_3 = tf.keras.layers.Dense(350, activation='relu', name='dense_3', dtype=tf.float64)(dense_2)
    dense_4 = tf.keras.layers.Dense(10, activation='softmax', name='dense_4', dtype=tf.float64)(dense_3)
    model = tf.keras.Model(inputs=[v for v in feature_layer_inputs.values()], outputs=dense_4)
    
    
    def my_acc(labels,probs):
        acc_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='my_acc')
        acc_metric.update_state(y_true=labels, y_pred=probs)
        return {'acc': acc_metric}
    
    
    def loss():
        return tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(labels,probs))
    

    
    with tf.GradientTape() as tape:
        
        tape.watch(model.trainable_variables)
        
        probs = model(features)
        class_index = tf.argmax(probs, axis=1 , output_type=tf.int64)
    
   
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            opt = tf.keras.optimizers.Adam()
            opt.iterations = tf.compat.v1.train.get_global_step()
            gradients = tape.gradient(loss(), model.trainable_variables)
            train_op = opt.apply_gradients(zip(gradients, model.trainable_variables))
            return tf.estimator.EstimatorSpec(mode, loss=loss(), train_op = train_op)
        
        
    if mode == tf.estimator.ModeKeys.EVAL:
        metrics = my_acc(labels,probs)
        return tf.estimator.EstimatorSpec(mode,loss=loss(), eval_metric_ops=metrics)



    elif mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'top_class_index' : class_index,
            'probs_of_classes' : probs
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

In [26]:
configurations = tf.estimator.RunConfig(save_summary_steps=35, save_checkpoints_steps=35, keep_checkpoint_max=1)
my_second_estimator = tf.estimator.Estimator(model_fn = my_model_fn, model_dir='../model/my_second_estimator', params=params, config=configurations)

INFO:tensorflow:Using config: {'_model_dir': '../model/my_second_estimator', '_tf_random_seed': None, '_save_summary_steps': 35, '_save_checkpoints_steps': 35, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [15]:
my_second_estimator.train(lambda:my_input_fn(dataframe=train_set, labels=train_y, batch_size=train_batch_size, train=True), steps=35)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model/my_second_estimator/model.ckpt-35
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 35 into ../model/my_second_estimator/model.ckpt.
INFO:tensorflow:loss = 1.0034117736484713, step = 35
INFO:tensorflow:Saving checkpoints for 70 into ../model/my_second_estimator/model.ckpt.
Instructions for updating:
Use standard file APIs to delete files with this prefix.
INFO:tensorflow:Loss for final step: 0.8390346226619476.


In [27]:
train_spec = tf.estimator.TrainSpec(input_fn=lambda:my_input_fn(train_set, train_y, train_batch_size, train=True), max_steps=35*params['num_epochs'])

eval_spec = tf.estimator.EvalSpec(input_fn=lambda:my_input_fn(valid_set, valid_y, valid_batch_size, train=False), steps=12)

tf.estimator.train_and_evaluate(my_second_estimator, train_spec, eval_spec)

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 35 or save_checkpoints_secs None.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ../model/my_second_estimator/model.ckpt.
INFO:tensorflow:loss = 2.2993057595780244, step = 0
INFO:tensorflow:Saving checkpoints for 35 into ..

({'acc': 0.5708333, 'loss': 1.0481333, 'global_step': 35}, [])

In [21]:
params

{'learning_rate': 0.001,
 'feature_names': ['fixed_acidity',
  'volatile_acidity',
  'citric_acid',
  'residual_sugar',
  'chlorides',
  'free_sulfur_dioxide',
  'total_sulfur_dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol'],
 'train_batch_size': 33,
 'valid_batch_size': 20,
 'test_batch_size': 17,
 'number_of_hidden_units': [200, 180, 10],
 'num_epochs': 5,
 'fixed_acidity': [15.6, 4.7],
 'volatile_acidity': [1.58, 0.12],
 'citric_acid': [1.0, 0.0],
 'residual_sugar': [15.5, 0.9],
 'chlorides': [0.61, 0.012],
 'free_sulfur_dioxide': [72.0, 1.0],
 'total_sulfur_dioxide': [289.0, 6.0],
 'density': [1.00369, 0.99007],
 'pH': [4.01, 2.74],
 'sulphates': [2.0, 0.33],
 'alcohol': [14.0, 8.4]}

## Serving

In [16]:
def my_serving_input_fn(my_params=params):
    feature_names = my_params['feature_names']
    receiver_tensors = {}
    input_feature = 'x'
    for names in feature_names:
        max_val = my_params[names][0]
        min_val = my_params[names][1]
        dif = max_val - min_val 
        receiver_tensors[names] = (tf.keras.backend.placeholder(shape=[None,1], dtype=tf.float32, name=names) - min_val)/dif  
    
    
    features = {
        input_feature: tf.concat([
            receiver_tensors['fixed_acidity'],
            receiver_tensors['volatile_acidity'],
            receiver_tensors['citric_acid'],
            receiver_tensors['residual_sugar'],
            receiver_tensors['chlorides'],
            receiver_tensors['free_sulfur_dioxide'],
            receiver_tensors['total_sulfur_dioxide'],
            receiver_tensors['density'],
            receiver_tensors['pH'],
            receiver_tensors['sulphates'],
            receiver_tensors['alcohol'],
        ], axis=1) 
    }
    return  tf.estimator.export.ServingInputReceiver(receiver_tensors=receiver_tensors, features=features)

In [19]:
export_dir = '../model/models/wine_model' #rename long number with number of version

serving_input_fn = my_serving_input_fn(my_params=params)

export_path = my_second_estimator.export_saved_model(
    export_dir_base=export_dir, serving_input_receiver_fn=lambda:my_serving_input_fn(params))

In [ ]:
docker run -p 8501:8501 --mount type=bind,source=/home/jpriest/Desktop/intro_to_estimators/model/models/wine_model/,target=/models/wine_model -e MODEL_NAME=wine_model -t tensorflow/serving


In [ ]:
curl -d @/home/jpriest/Desktop/instance.json -X POST http://localhost:8501/v1/models/wine_model:predict